In [81]:
library(bnlearn)
library(Rgraphviz)
library(parallel)
library(tidyverse)
suppressWarnings(library(dplyr))

### dataset needs to be preprocessed because  for example had values as factors. My functions does not work w/factorslearning.test

Today i learnt that the k2 algorithm infers the dag, the directed acyclic graph, so the structure of the bayesian network from the dataset. Not from the fitted model. 
What i initially thought was that you happen to have a model and you want to reverse engineer that. 

In [82]:
factor_remover <- function(dataset) {
    cols <- colnames(dataset)
    for (i in seq_along(cols)) {
        name <- cols[i]
        dataset[[name]] <- as.character(dataset[[name]])
    }
    return(dataset)
}

node.values <- function(dataset, parents) {
    # retrieves the unique values of the nodes
    #return a list of vectors, 
    # each vector contains the unique values of the corresponding node
    output <- list()
    for (kk in seq_along(parents)) {
        node <- parents[kk]
        node.vals <- unique(dataset[[node]]) #it was levels()
        output[[kk]] <- node.vals
    }
    return(output)
}

old.alphaijk <- function(i, pii, dataset, nodes.order) {
    actual_node <- nodes.order[i]
    all.nodes <- c(pii, actual_node)
    subset <- dataset[all.nodes]
    unique.instantiations <- expand.grid(node.values(dataset, all.nodes))
    output <- c()
    for (tracker in 1:dim(unique.instantiations)[1]){
        aa <- 0
        for (j in 1:dim(subset)[1]) {
            ifelse(all(as.vector(unlist(subset[j,])) == as.vector(unlist(unique.instantiations[tracker,]))), aa <- aa +1, FALSE)
        }
        output <- c(output,aa)
    }
    return(output)
}

fastalphaijk <- function(i, pii, dataset, nodes.order) {
    actual_node <- nodes.order[i]
    all.nodes <- c(pii, actual_node)
    subset <- dataset[all.nodes]
    unique.instantiations <- expand.grid(node.values(dataset, all.nodes))
    colnames(unique.instantiations) <- colnames(subset)
    subset.counted <- subset |> group_by(across(everything())) |> summarise(n = n(), .groups="drop")
    checking <-  left_join(unique.instantiations, subset.counted, by=colnames(subset), na_matches="na")  |> arrange(across(-last_col()))
    checking[is.na(checking)]<- 0
    #print(checking)
    return(checking$n)
}


k2helper <- function(i, pii, dataset, nodes.order) {
    # the "f" function in the pdf
    phi_i <- expand.grid(node.values(dataset, pii)) 
    #cat("\nphi.i")
    #print( phi_i)
    q_i <- dim(phi_i)[1] # works good
    #cat("\nqi.i", q_i)
    nodes <- colnames(dataset)
    #cat("\nnodes", nodes)
    i.th.node <- nodes[i]
    #cat("\nithnode", i.th.node)
    v_i <- node.values(dataset, i.th.node)
    #cat("\nvi")
    #print( v_i)
    r_i <- length(v_i[[1]])
    #cat("ri", r_i)
    individual_alphas <- fastalphaijk(i, pii, dataset, nodes.order) # works good
    #cat("\nindividual alphas::",individual_alphas)
    aijk <- prod(factorial(individual_alphas)) ##
    #numeratore <- factorial(r_i-1)
    grouping_alphas <- matrix(individual_alphas, nrow=r_i) # to get the right Nijs
    nij <- colSums(grouping_alphas)
    #print(nij)
    produttoria_alpha <- prod(factorial(individual_alphas))
    numeratore <- factorial(r_i-1)
    denominatore <- factorial(nij+r_i-1)
    out <- prod((numeratore/denominatore))*produttoria_alpha
    #cat("\nproduttoria_alpha",produttoria_alpha )
    #cat("\nproduttoria_alpha",produttoria_alpha )
    #print(frazione)
    return(out)
}


k2 <- function(dataset, nodes.order, upperbound) {
    nodes <- colnames(dataset)
    for (i in seq_along(nodes.order)) {
        actual_node <- nodes.order[i]
        #cat("\n", i,"\n")
        pii <- c()
        p.old <- k2helper(i, pii,dataset, nodes.order)
        #cat("\n ",p.old, "\n should be ", 1/2772)
        proceed <- TRUE
        cat("\ni::",i, "\n")
        while (proceed && length(pii) < upperbound) {
            predecessors_idx <- i-1
            predecessors <- nodes.order[0:predecessors_idx]
            cat("\npredecessors::", predecessors, " pii:: ", pii)
            piiuz <- setdiff(predecessors, pii)
            if (length(piiuz) > 1) {
                daddy.s.probs <- c()
                for (daddy in seq_along(piiuz)) {
                    p.new <- k2helper(i, piiuz[daddy], dataset, nodes.order)
                    daddy.s.probs <- c(daddy.s.probs, p.new)
                }
                p.new <- max(daddy.s.probs)
                genitore <-  piiuz[daddy.s.probs == p.new]
            } else {
                p.new <- k2helper(i, piiuz, dataset, nodes.order)
                genitore <- piiuz
            }
            cat("\npiiuz:: ",piiuz,"\np.old:: ", p.old, "\np.new:: ", p.new)
            #roof <- roof+1
            if (p.new > p.old) {
                p.old <- p.new
                pii <- c(pii, genitore)
                #ifelse(roof>length(nodes)^length(nodes), proceed <- FALSE, proceed <- TRUE)
            } else  {
                proceed <- FALSE
            }
        }
        cat('\nNode', nodes.order[i], '--> Parents:', pii,'\n') 
    }
    return(TRUE)
}

In [87]:
###Trial with Paper dataset
#
#df <- data.frame(x1=c(1,1,0,1,0,0,1,0,1,0), x2=c(0,1,0,1,0,1,1,0,1,0), x3=c(0,1,1,1,0,1,1,0,1,0))
#k2(dataset=df, upperbound=2, nodes.order=colnames(df))


In [ ]:

## Trial w/ learning.test
df <- learning.test
#print(sapply(df, class))
df <- factor_remover(df)
#print(sapply(df, class))

Warning message in data(child):
“data set ‘child’ not found”


In [91]:
?bnlearn

bnlearn-package            package:bnlearn             R Documentation

_B_a_y_e_s_i_a_n _n_e_t_w_o_r_k _s_t_r_u_c_t_u_r_e _l_e_a_r_n_i_n_g, _p_a_r_a_m_e_t_e_r _l_e_a_r_n_i_n_g _a_n_d _i_n_f_e_r_e_n_c_e

_D_e_s_c_r_i_p_t_i_o_n:

     Bayesian network structure learning (via constraint-based,
     score-based and hybrid algorithms), parameter learning (via ML and
     Bayesian estimators) and inference (via approximate inference
     algorithms).

_D_e_t_a_i_l_s:

     ‘bnlearn’ implements key algorithms covering all stages of
     Bayesian network modelling: data preprocessing, structure learning
     combining data and expert/prior knowledge, parameter learning, and
     inference (including causal inference via do-calculus). ‘bnlearn’
     aims to be a one-stop shop for Bayesian networks in R, providing
     the tools needed for learning and working with discrete Bayesian
     networks, Gaussian Bayesian n